# Python client for the Glowing Bear data warehouse
---------------

Interacting with the data in the [tranSMART Glowing Bear data warehouse](https://glowingbear.app) via the [REST API](https://glowingbear.app/docs/technical/#transmart-api) and the [Python client](https://github.com/thehyve/transmart-api-client-py) into the Jupyter Notebook analytical environment.

## Getting started

* Make sure you have registered a free account for the [The Hyve public demonstration environment of Glowing Bear](https://glowingbear.app/getting-started/). 
* Make sure you install the Python client with `pip install transmart[full]` to have all functionality available. (When using this notebook in Binder this has already been done for you)
* If you'd like more information or a tailor-made demonstration, please [reach out to The Hyve](https://thehyve.nl/contact/).

First we will import the [tranSMART Python package](https://github.com/thehyve/transmart-api-client-py) by executing the following cell. To execute a cell, select it and press the Run button above.

In [ ]:
import transmart as tm

We'll also import some other libraries, to help us later:

In [ ]:
import json
import pandas as pd

from math import pi

from bokeh.io import output_notebook, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum

output_notebook()

# Connecting to the tranSMART server
Now we will authenticate to the tranSMART server with your credentials. Just execute the following cell, fill in the details and press Enter:

Generate an offline token for your user:

In [ ]:
import requests

# Keycloak credentials you also use to access Glowing Bear
user = "ewelina"
password = "1234"
kc_token_url = "https://keycloak-dwh-test.thehyve.net/auth/realms/transmart/protocol/openid-connect/token"
kc_client_id= "transmart-client"

r = requests.post(url=kc_token_url,
                  data=dict(grant_type='password',
                            client_id=kc_client_id,
                            scope= 'offline_access',
                            username=user,
                            password=password
                           )
                 )
offline_token = r.json().get('refresh_token')
offline_token

In [ ]:
# Create an API object to perform API queries with, using our user credentials
api = tm.get_api(
    host = 'https://transmart.thehyve.net', # URL of tranSMART server connected to your Glowing Bear
    kc_url = "https://keycloak-dwh-test.thehyve.net", # URL of Keycloak connected to your Glowing Bear
    kc_realm = "transmart", # Realm in Keycloak for the tranSMART application
    offline_token = offline_token,
    print_urls = False # Whether or not to print the API URLs used behind the scenes, to learn the API calls.
)

# Common errors:
# * '401 Client Error: Unauthorized' - Wrong username/password
# * 'HTTPSConnectionPool' - Wrong tranSMART or Keycloak URL or no internet
# * '404 Client Error: Not Found' - Wrong Keycloak realm

# Exploring the data
A tranSMART Glowing Bear server can contain multiple data sets or studies, which can be used to control access to per user. Let's show all sets in the server that our user has access to:

In [ ]:
# Query for the available studies/data sets and show the first five
studies = api.get_studies()
studies.dataframe.head()

The most important data overview in tranSMART Glowing Bear is the tree. Let's see the highest levels of the tree, including patient counts:

In [ ]:
# Query and print the first 2 levels of the tree, including the patient counts
tree = api.tree_nodes(depth=2, counts=True)
tree

When we want more details on the tree nodes (like which concept is behind it) we can use the dataframe representation instead:

In [ ]:
# Show the first five rows of the tree dataframe
tree.dataframe.head()

# Querying for patients and observations
The following example queries for recent, normal pregnancies from parents born in Boston or Cambridge, MA.  
(Later we'll explore in more detail how to build such a query from scratch)

In [ ]:
# Construct the query
pregnancy_constraint = \
    api.new_constraint(concept='Demographics:BIRTHPLACE', value_list=['Boston MA US', 'Cambridge MA US']) & \
    api.new_constraint(concept='Conditions-SNOMED:72892002', min_start_date='2009-01-01')

# Calculate and print the counts
output = api.observations.counts(constraint=pregnancy_constraint)
print(json.dumps(output, indent=2))

With the above call we can quickly see the number of matching patients and the total number of observations (not just pregnancy observations) linked to these subjects, without needing to retrieve the data.

Next, we retrieve the matching patients and show the details for the first five:

In [ ]:
# Query and show the first five patients matching the query
api.patients(constraint=pregnancy_constraint).dataframe.head()

Let's also show the first five observations linked to this patient set:

In [ ]:
# Query and show the first five observations matching the query
api.observations(constraint=pregnancy_constraint).dataframe.head()

# Saving a Patient Set
Since we want to reuse this patient set, let's store it on the server and retrieve the returned patient set ID:

In [ ]:
# Creating the patient set and store and print the returned patient set ID
patient_set_id = api.create_patient_set(
    name="Recent pregnancies in Boston and Cambridge", constraint=pregnancy_constraint)['id']
print("Patient Set ID for new patient set: {}".format(patient_set_id))

Let's show the most recent stored patient sets linked to my user account, to see our newest included:

In [ ]:
# Query all our patient sets and display the five most recent sets
api.patient_sets().dataframe.sort_values(by='id', axis=0).tail()

# Note: Currently only patient sets saved via the API or those used in the Cross Table in Glowing Bear are shown

We can now use this patient set ID to continue making queries. For example, with the following query that requests only the Ethnicity data for our selected subjects:

In [ ]:
# Create new query constraint for observations (and linked patients) where the patient is in the patientset
# and the concept is Ethnicity
one_concept_constraint = api.new_constraint(subject_set_id=patient_set_id, concept='Demographics:ETHNICITY')

# Retrieve the matching observations and display the first five
api.observations(constraint=one_concept_constraint).dataframe.head()

Getting the data to Python or R also allows us to easily make interesting figures or analyses, like the below pie chart of the Ethnicity distribution in our selected cohort:

In [ ]:
# Retrieving the summarized count data for Ethnicity for our query constraint
aggregates = api.observations.aggregates_per_concept(constraint=one_concept_constraint)
x = aggregates['aggregatesPerConcept']['Demographics:ETHNICITY']['categoricalValueAggregates']['valueCounts']

# Setting the Bokeh plot data and variables
data = pd.Series(x).reset_index(name='value').rename(columns={'index':'ethnicity'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]
p = figure(plot_height=350, title="Ethnicity distribution for recent parents in Boston or Cambridge, MA", toolbar_location=None,
           tools="hover", tooltips="@ethnicity: @value", x_range=(-0.5, 1.0))
p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='ethnicity', source=data)
p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None
show(p)

# Constructing a query
Below we will, step by step, create the query for: _Men with Type 2 Diabetes_.

If a concept is easily found by browsing the tree, like Gender under Demographics, we can start with that:

In [ ]:
# Retrieve and display the first two levels of the tree under the Demographics top node
tree = api.tree_nodes(depth=2, root='\\Demographics\\')
tree.dataframe

We see that the concept code for Gender is `Demographics:GENDER`, which we will use to start our query constraint:

In [ ]:
# Creating the constraint to limit to Gender observations
gender_constraint = api.new_constraint(concept='Demographics:GENDER')

# Printing the type of the constraint, to see this is an ObservationConstraint
print(type(gender_constraint))
print()

# Showing the definition of our constraint as a dictionary overview
output = gender_constraint
print(json.dumps(json.loads(str(output)), indent=2))

We see that the type of the constraint is an `ObservationConstraint`. And below that we see the representation of our current query.

All parts of our query that need to be true within the same observation will need to be added to the same `ObservationConstraint`. For the _Men_ part of our query both concept Gender and value Male need to be true for the same observation, so they should be together in one `ObservationConstraint`.  
The _Type 2 Diabetes_ will need to be a separate observation for the same patients, so we will later create a separate `ObservationConstraint` for that.

Now, we're not just interested in anyone with a recorded Gender, but only those were the value for this is Male. Let's see what the possible values for Gender are:

In [ ]:
# Retrieve the aggregate counts for the Gender concept and print the possible values
output = api.observations.aggregates_per_concept(constraint=gender_constraint)
print(json.dumps(output['aggregatesPerConcept']['Demographics:GENDER']['categoricalValueAggregates'], indent=2))

We see that the possible values are F and M, where we are interested in the subjects with value M. For this we need to restrict our current constraint with the value M.

We can see all possible parameters for our `ObservationConstraint` below:

In [ ]:
# Print all the possible parameters for a query constraint
for key in api.new_constraint().params:
    print("* {}".format(key))

For restricting our categorical value we need to use `value_list` with a list including the value 'M':

In [ ]:
# Adding the value_list constraint to our query constraint, to limit on observations of gender Male
gender_constraint.value_list = ['M']

# Print a representation of our constraint
output = gender_constraint
print(json.dumps(json.loads(str(output)), indent=2))

We can see in the above representation that we are searching for all observations that are both linked to the Concept Gender AND has the value 'M'. We will thus find all observations of Gender male and the patients linked to those observations.

Let's count how many observations and patients match this query:

In [ ]:
# Retrieve and print the counts for observations and patients matching our constraint
output = api.observations.counts(constraint=gender_constraint)
print(json.dumps(output, indent=2))

The patient count indeed represents the number of male subjects in our study.

Note that the observation count is not the the number of ALL observations linked to male patients, but only the observations of concept Gender with Value male.

If we instead want to treat our query as a patient set, and want to know how many observations in total are linked to these patients we have to wrap our query in a patient subselection constraint:

In [ ]:
# Adding the subselection parameter to our query constraint, to treat the selection as a patient set
gender_constraint.subselection = 'patient'

# Print a representation of our constraint
output = gender_constraint
print(json.dumps(json.loads(str(output)), indent=2))

We will now see that the observation count for this query is indeed much higher:

In [ ]:
# Retrieve and print the counts for observations and patients matching our Male Gender patient set constraint
output = api.observations.counts(constraint=gender_constraint)
print(json.dumps(output, indent=2))

Our first `ObservationConstraint` for the _Male_ part of our question is done. Now, we need to find what the tree node or concept code is for _Type 2 Diabetes_.

We know we have used SNOMED (and ICD10) to structure the data in our tranSMART server. In [Bioportal we find](https://bioportal.bioontology.org/ontologies/SNOMEDCT?p=classes&conceptid=44054006) that the SNOMED code (notation) for 'Type 2 diabetes mellitus' is 44054006. So let's search our tree for that and get the node details:

In [ ]:
# Search the tree for our SNOMED code
nodes_with_snomed_code = api.search_tree_node('name:44054006')

# Retrieve the details for the first matching tree node
node_details = api.tree_dict[nodes_with_snomed_code[0]]

# Print a representation of the tree node
print(json.dumps(node_details, indent=2))

Here, we learn that the concept code in tranSMART Glowing Bear is `Conditions-SNOMED:44054006`, which we will use to construct our second `ObservationConstraint`:

In [ ]:
# Create a new query constraint filtering on the concept code found in the tree node 
diabetes_constraint = api.new_constraint(
    concept=node_details['constraint.conceptCode'], subselection='patient')

# Print a representation of our constraint
print(json.dumps(json.loads(str(diabetes_constraint)), indent=2))

And the count:

In [ ]:
# Retrieve and print the counts for observations and patients matching our Diabetes constraint
output = api.observations.counts(constraint=diabetes_constraint)
print(json.dumps(output, indent=2))

Now we have also finished the second part of our query. We are now looking for the intersection between the two patient sets we have made.

For this we can simply use the `&` or `|` operators and brackets to combine multiple `ObservationConstraints` into one `GroupConstraint`:

In [ ]:
# Creating a group constraint by combining the Male Gender and Diabetes constraints with the 'AND' operator
men_with_diabetes_constraint = gender_constraint & diabetes_constraint

# Printing the type of the constraint, to see this is an GroupConstraint
print(type(men_with_diabetes_constraint))
print()

# Print a representation of our constraint
print(json.dumps(json.loads(str(men_with_diabetes_constraint)), indent=2))

Side note: When creating a `GroupConstraint`, the multiple `ObservationConstraint`s are automatically each wrapped with a patient subselection. So technically it wasn't necessary that we did this ourselves above for the two constraints.

Now all that's left is for us to count our final query and retrieve the patients and observations of interest!

In [ ]:
# Retrieve and print the counts for observations and patients matching our Men with Diabetes query constraint
output = api.observations.counts(constraint=men_with_diabetes_constraint)
print(json.dumps(output, indent=2))

# Retrieve the patients matching our constraint and displaying the first five
display(api.patients(constraint=men_with_diabetes_constraint).dataframe.head())

# Retrieve the observations matching our constraint and displaying the first five
display(api.observations(constraint=men_with_diabetes_constraint).dataframe.head())